<a href="https://colab.research.google.com/github/KarnB013/scamzeroed/blob/main/scamzeroed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=c4f0072d42f0b64d5475573c7881dc7ab01e3219b840f539cd406abbb3203c5a
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_7657ABH5Q5ZXF4FU4L6ARAWA-GY4HFQS5AZZHVPRI")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
pip install rake-nltk

In [ ]:
from rake_nltk import Rake
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
r = Rake()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import requests
import json
import urllib
from urllib.parse import quote
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

In [ ]:
df = pd.read_table("SMSSpamCollection" , sep='\t' ,header =None)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)

df

In [ ]:
df[0] = df[0].replace('ham', 0)
df[0] = df[0].replace('spam', 1)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[1], df[0], test_size=0.2, random_state=42)

count_vector = CountVectorizer(ngram_range=(1, 1), lowercase = True , stop_words =  'english')
X_train_count = count_vector.fit_transform(X_train)
X_test_count = count_vector.transform(X_test)
X_train_feature_list = count_vector.get_feature_names_out()

X_train_feature_list

In [ ]:
naive_bayes = MultinomialNB()
y_train_reshape = y_train.array.reshape(-1, 1)
naive_bayes.fit(X_train_count , y_train_reshape)
predictions = naive_bayes.predict(X_test_count)

predictions

In [ ]:
print('Accuracy score: ', format(accuracy_score(predictions, y_test)))
print('Precision score: ', format(precision_score(predictions, y_test)))
print('Recall score: ', format(recall_score(predictions, y_test)))
print('F1 score: ', format(f1_score(predictions, y_test)))

In [ ]:
# The SVC had worse performance compared to the Naive Bayes Classifiers

naive_bayes = SVC()
y_train = y_train.array.reshape(-1, 1)
naive_bayes.fit(X_train_count , y_train_reshape)
predictions = naive_bayes.predict(X_test_count)

In [26]:
# with open('model.pkl','wb') as f:
#     pickle.dump(naive_bayes,f)

with open('model.pkl', 'rb') as f:
    naive_bayes2 = pickle.load(f)

In [27]:
# with open('count_vector.pkl', 'wb') as f:
#     pickle.dump(count_vector, f)

with open('count_vector.pkl', 'rb') as f:
    count_vector2 = pickle.load(f)

In [28]:
@anvil.server.callable
def detect_scam_words(text):
  predictors = count_vector.transform(text)
  result = naive_bayes.predict(predictors)
  print(result)
  average = sum(result)/len(result)
  print(average)
  if average >= 0.5:
    return True
  else:
    return False

In [ ]:
@anvil.server.callable
def keyword(key):
  r.extract_keywords_from_text(key)
  a = r.get_ranked_phrases()
  return a


In [ ]:
@anvil.server.callable
def url_extract(url_query):
  url=r"\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b"
  check_url = re.findall(url, url_query)
  return check_url

In [ ]:
@anvil.server.callable
def is_scam(url_query):
  for i in url_query:
    ii = i.replace("/", "%2F")
    req = "https://www.ipqualityscore.com/api/json/url/your_private_key/"+quote(ii)
    resp = requests.get(req)
    print(resp.json())
    if(resp.status_code == 200):
      spam = resp.json()['spamming']
      malware = resp.json()['malware']
      sus = resp.json()['suspicious']
      phishing = resp.json()['phishing']
      unsafe = resp.json()['unsafe']
      risk = resp.json()['risk_score']

    if(spam or malware or sus or phishing or unsafe or risk > 75):
      return True
    else:
      return False

In [25]:
anvil.server.wait_forever()

{'message': 'Success.', 'success': True, 'unsafe': False, 'domain': 'scamsite.me', 'root_domain': 'scamsite.me', 'server': 'N/A', 'content_type': '', 'status_code': 0, 'page_size': 0, 'domain_rank': 0, 'dns_valid': False, 'parking': False, 'spamming': False, 'malware': False, 'phishing': False, 'suspicious': True, 'adult': False, 'risk_score': 92, 'country_code': 'N/A', 'language_code': 'N/A', 'category': 'N/A', 'domain_age': {'human': 'just now', 'timestamp': 1708288285, 'iso': '2024-02-18T15:31:25-05:00'}, 'domain_trust': 'Upgraded plan required.', 'redirected': False, 'short_link_redirect': False, 'hosted_content': False, 'page_title': 'N/A', 'risky_tld': False, 'spf_record': False, 'dmarc_record': False, 'technologies': [], 'a_records': [], 'mx_records': [], 'ns_records': [], 'ip_address': 'N/A', 'final_url': 'http://scamsite.me', 'request_id': 'L9Urx7Duye'}
[0]
0.0
{'message': 'Success.', 'success': True, 'unsafe': False, 'domain': 'scam.de', 'root_domain': 'scam.de', 'ip_address'

KeyboardInterrupt: 